# Добро пожаловать в самостоятельный проект

Самостоятельный проект — это практическая проверка знаний, приобретённых вами на вводном курсе. Каждый раздел посвящён отдельной стадии анализа данных с экскурсом в основы Python. 

Проект выполняется в пять этапов:

•	Постановка задачи

•	Получение данных

•	Предобработка данных

•	Анализ данных

•	Оформление результатов

Для каждой части описаны шаги выполнения c теоретическим приложением. В Jupyter Notebook эти шаги связаны между собой выводами и результатами.

**Исследование: Музыка больших городов**

Яндекс.Музыка — это крупный продукт с огромным запасом данных для исследований. Команды таких сервисов для поддержания интереса к продукту и привлечения новых пользователей часто проводят исследования про пользователей. Чтобы удержать клиентов и привлечь новых, сделать бренд более узнаваемым, команда сервиса проводит исследования аудитории, и публикует интересные результаты. Например, интересно сравнить тексты, сочинённые нейросетью, с произведениями настоящих рэперов.
Есть исследование, которое напоминает наше: о музыкальных предпочтениях в разных городах России.
Итак, вопрос вам: как музыка, которая звучит по дороге на работу в понедельник утром, отличается от той, что играет в среду или в конце рабочей недели? Возьмите данные для Москвы и Петербурга. Сравните, что и в каком режиме слушают их жители.

План исследования

1.	Получение данных. Прочитайте данные, ознакомьтесь с ними.

2.	Предобработка данных. Избавьтесь от дубликатов, проблем с названиями столбцов и пропусками.

3.	Анализ данных. Ответьте на основные вопросы исследования, подготовьте отчётную таблицу или опишите полученный результат.

4.	Подведение итогов. Просмотрите выполненную работу и сформулируйте выводы.


# Этап 1. Получение данных

Изучим данные, предоставленные сервисом для проекта.

Прочитайте файл music_project.csv и сохраните его в переменной df.

Получите  первых 10 строк таблицы, а также общую информацию о данных таблицы df.

In [850]:
import pandas as pd
df = pd.read_csv('yandex_music.csv')
print(df.head(10))
print(df.info())


     userID                        Track            artist   genre  \
0  FFB692EC            Kamigata To Boots  The Mass Missile    rock   
1  55204538  Delayed Because of Accident  Andreas Rönnberg    rock   
2    20EC38            Funiculì funiculà       Mario Lanza     pop   
3  A3DD03C9        Dragons in the Sunset        Fire + Ice    folk   
4  E2DC1FAE                  Soul People        Space Echo   dance   
5  842029A1                    Преданная         IMPERVTOR  rusrap   
6  4CB90AA5                         True      Roman Messer   dance   
7  F03E1C1F             Feeling This Way   Polina Griffith   dance   
8  8FA1D3BE     И вновь продолжается бой               NaN  ruspop   
9  E772D5C0                    Pessimist               NaN   dance   

             City        time        Day  
0  Saint-Petersburg  20:28:33  Wednesday  
1            Moscow  14:07:09     Friday  
2  Saint-Petersburg  20:58:07  Wednesday  
3  Saint-Petersburg  08:37:09     Monday  
4            M

Рассмотрим полученную информацию подробнее.
Всего в таблице 7 столбцов, тип данных у каждого столбца - строка.
Подробно разберём, какие в df столбцы и какую информацию они содержат:
•	userID — идентификатор пользователя;
•	Track — название трека;
•	artist — имя исполнителя;
•	genre — название жанра;
•	City — город, в котором происходило прослушивание;
•	time — время, в которое пользователь слушал трек;
•	Day — день недели.
Количество значений в столбцах различается. Это говорит о том, что в данных есть пропущенные значения.


## Выводы: 

Каждая строка таблицы содержит информацию о композициях определённого жанра в определённом исполнении, которые пользователи слушали в одном из городов в определённое время и день недели. Две проблемы, которые нужно решать: пропуски и некачественные названия столбцов. Для проверки рабочих гипотез особенно ценны столбцы genre и time. Данные из столбца genre  позволят узнать самые популярные жанры.

# Этап 2. Предобработка данных

Исключим пропуски, переименуем столбцы, а также проверим данные на наличие дубликатов.

Получаем перечень названий столбцов. Какая наблюдается проблема — кроме тех, что уже были названы ранее?


In [851]:
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

В названиях столбцов есть пробелы, которые могут затруднять доступ к данным.

Переименуем столбцы для удобства дальнейшей работы. Проверим результат.


In [852]:
df.rename(columns={'  userID':'user_ID', 'Track':'track_name', 'artist':'artist_name', '  City  ':'City', 'genre':'genre_name', 'Day':'weekday'}, inplace=True)
df.columns

Index(['user_ID', 'track_name', 'artist_name', 'genre_name', 'City', 'time',
       'weekday'],
      dtype='object')

Проверим данные на наличие пропусков вызовом набора методов для суммирования пропущенных значений.

In [853]:
df.isnull().sum()

user_ID           0
track_name     1231
artist_name    7203
genre_name     1198
City              0
time              0
weekday           0
dtype: int64

Пустые значения свидетельствуют, что для некоторых треков доступна не вся информация. Причины могут быть разные: скажем, не назван конкретный исполнитель народной песни. Хуже, если проблемы с записью данных. Каждый отдельный случай необходимо разобрать и выявить причину.

Заменяем пропущенные значения в столбцах с названием трека и исполнителя на строку 'unknown'. После этой операции нужно убедиться, что таблица больше не содержит пропусков.


In [854]:
df['track_name'] = df['track_name'].fillna(value='unknown')
df['artist_name'] = df['artist_name'].fillna(value='unknown')

In [855]:
df.isnull().sum()

user_ID           0
track_name        0
artist_name       0
genre_name     1198
City              0
time              0
weekday           0
dtype: int64

Удаляем в столбце с жанрами пустые значения; убеждаемся, что их больше не осталось.

In [856]:
df.dropna(subset='genre_name', inplace=True)
df.isna().sum()

user_ID        0
track_name     0
artist_name    0
genre_name     0
City           0
time           0
weekday        0
dtype: int64

Необходимо установить наличие дубликатов. Если найдутся, удаляем, и проверяем, все ли удалились.

In [857]:
print(df.duplicated().sum())
df.drop_duplicates(inplace=True)
print(df.duplicated().sum())

3755
0


Дубликаты могли появиться вследствие сбоя в записи данных. Стоит обратить внимание и разобраться с причинами появления такого «информационного мусора».

Сохраняем список уникальных значений столбца с жанрами в переменной genres_list.

Объявим функцию find_genre() для поиска неявных дубликатов в столбце с жанрами. Например, когда название одного и того же жанра написано разными словами.


In [858]:
genres_list = df['genre_name'].unique()

In [859]:
def find_genre(key_genre):
	count = 0
	for genre in genres_list:
		if genre == key_genre:
			count+=1
	return count


Вызов функции find_genre() для поиска различных вариантов названия жанра хип-хоп в таблице.

Правильное название — hiphop. Поищем другие варианты:

•	hip

•	hop

•	hip-hop


In [860]:
print(find_genre('hip'))

1


In [861]:
print(find_genre('hop'))

1


In [862]:
print(find_genre('hip-hop'))

1


Объявим функцию find_hip_hop(), которая заменяет неправильное название этого жанра в столбце 'genre_name' на 'hiphop' и проверяет успешность выполнения замены.

Так исправляем все варианты написания, которые выявила проверка.


In [863]:
def find_hip_hop(table, to_change):
	table.replace(to_change, 'hiphop', inplace=True)
	return table[table['genre_name'] == f'{to_change}']['genre_name'].count()
# функция принимает как параметры таблицу df и неверное название
# к столбцу 'genre_name' применяется специальный метод, 
# который заменяет второй параметр на строку 'hiphop'
# результат работы равен подсчитанному методом count() числу
#значений столбца, 
# которые равны второму параметру
# функция возвращает результат


In [864]:
print(find_hip_hop(df, 'hip'))
print(find_hip_hop(df, 'hop'))
print(find_hip_hop(df, 'hip-hop'))
print(df[df['genre_name'] == 'hiphop']['genre_name'].count())

0
0
0
3056


Получаем общую информацию о данных. Убеждаемся, что чистка выполнена успешно.

In [865]:
print(df.info())


<class 'pandas.core.frame.DataFrame'>
Index: 60126 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_ID      60126 non-null  object
 1   track_name   60126 non-null  object
 2   artist_name  60126 non-null  object
 3   genre_name   60126 non-null  object
 4   City         60126 non-null  object
 5   time         60126 non-null  object
 6   weekday      60126 non-null  object
dtypes: object(7)
memory usage: 3.7+ MB
None


## Вывод

На этапе предобработки в данных обнаружились не только пропуски и проблемы с названиями столбцов, но и всяческие виды дубликатов. Их удаление позволит провести анализ точнее. Поскольку сведения о жанрах важно сохранить для анализа, не просто удаляем все пропущенные значения, но заполним пропущенные имена исполнителей и названия треков. Имена столбцов теперь корректны и удобны для дальнейшей работы.


# Действительно ли музыку в разных городах слушают по-разному?

Была выдвинута гипотеза, что в Москве и Санкт-Петербурге пользователи слушают музыку по-разному. Проверяем это предположение по данным о трёх днях недели — понедельнике, среде и пятнице.

Для каждого города устанавливаем количество прослушанных в эти дни композиций с известным жанром, и сравниваем результаты.
Группируем данные по городу и вызовом метода count() подсчитываем композиции, для которых известен жанр.


In [866]:
df.groupby(by='City')['genre_name'].count()

City
Moscow              41892
Saint-Petersburg    18234
Name: genre_name, dtype: int64

В Москве прослушиваний больше, чем в Санкт-Петербург, но это не значит, что Москва более активна. У Яндекс.Музыки в целом больше пользователей в Москве, поэтому величины сопоставимы.
Сгруппируем данные по дню недели и подсчитаем прослушанные в понедельник, среду и пятницу композиции, для которых известен жанр.


In [867]:
df.groupby(by='weekday')['genre_name'].count()

weekday
Friday       21482
Monday       20866
Wednesday    17778
Name: genre_name, dtype: int64

Понедельник и пятница — время для музыки; по средам пользователи немного больше вовлечены в работу.

Создаём функцию number_tracks(), которая принимает как параметры таблицу, день недели и название города, а возвращает количество прослушанных композиций, для которых известен жанр. Проверяем количество прослушанных композиций для каждого города и понедельника, затем среды и пятницы.


In [868]:
def number_tracks(df, day, city):
	track_list = df[(df['weekday'] == day) & (df['City'] == city)]
	track_list_count = track_list['genre_name'].count()
	return track_list_count


In [869]:
Moscow_monday=number_tracks(df, 'Monday', 'Moscow')
print(Moscow_monday)

15347


In [870]:
Petersburg_monday=number_tracks(df, 'Monday', 'Saint-Petersburg')
print(Petersburg_monday)

5519


In [871]:
Moscow_wednesday=number_tracks(df, 'Wednesday', 'Moscow')
print(Moscow_wednesday)

10865


In [872]:
Petersburg_wednesday=number_tracks(df, 'Wednesday', 'Saint-Petersburg')
print(Petersburg_wednesday)

6913


In [873]:
Moscow_friday=number_tracks(df, 'Friday', 'Moscow')
print(Moscow_friday)

15680


In [874]:
Petersburg_friday=number_tracks(df, 'Friday', 'Saint-Petersburg')
print(Petersburg_friday)

5802


Сведём полученную информацию в одну таблицу, где ['city', 'monday', 'wednesday', 'friday'] названия столбцов.

In [875]:
result = pd.DataFrame({'city':['Moscow', 'Saints-Petersburg'], 'monday':[Moscow_monday, Petersburg_monday], 'wednesday':[Moscow_wednesday, Petersburg_wednesday], 'friday':[Moscow_friday, Petersburg_friday]})
result

,city,monday,wednesday,friday
0,Moscow,15347,10865,15680
1,Saints-Petersburg,5519,6913,5802


## Вывод

*Запишите здесь ваши выводы.Опишите закономерности, которые вы наблюдаете по дням недели и городам*

Москва лидирует по количеству прослушиваний каждый день недели. Наибольшее количество прослушиваний в Москве приходится на пятницу с небольшим отрывом от среды, а в Санкт Петербурге на среду.

# Утро понедельника и вечер пятницы — разная музыка или одна и та же?

Ищем ответ на вопрос, какие жанры преобладают в разных городах в понедельник утром и в пятницу вечером. Есть предположение, что в понедельник утром пользователи слушают больше бодрящей музыки (например, жанра поп), а вечером пятницы — больше танцевальных (например, электронику).
Получим таблицы данных по Москве moscow_general и по Санкт-Петербургу spb_general.


In [876]:
moscow_general = df[df['City'] == 'Moscow']
moscow_general['time']=pd.to_datetime(moscow_general['time'])
spb_general = df[df['City'] == 'Saint-Petersburg']
spb_general['time']=pd.to_datetime(spb_general['time'])

C:\Users\asustud\AppData\Local\Temp\ipykernel_1840\2747503104.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  moscow_general['time']=pd.to_datetime(moscow_general['time'])
C:\Users\asustud\AppData\Local\Temp\ipykernel_1840\2747503104.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moscow_general['time']=pd.to_datetime(moscow_general['time'])
C:\Users\asustud\AppData\Local\Temp\ipykernel_1840\2747503104.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  spb_genera

Создаём функцию genre_weekday(), которая возвращает список жанров по запрошенному дню недели и времени суток с такого-то часа по такой-то.

In [877]:
def genre_weekday(df, day, time1, time2):
	genre_list = df[(df['weekday']==day)&(df['time']>time1)&(df['time']<time2)]
	genre_list_sorted = genre_list.groupby('genre_name')['genre_name'].count().sort_values(ascending=False).head(10)
	return genre_list_sorted

Cравниваем полученные результаты по таблице для Москвы и Санкт-Петербурга в понедельник утром (с 7 до 11) и в пятницу вечером (с 17 до 23).

In [878]:
print(genre_weekday(moscow_general, 'Monday', '7:00:00', '11:00:00'))

genre_name
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
classical      157
Name: genre_name, dtype: int64


In [879]:
print(genre_weekday(spb_general, 'Monday', '7:00:00', '11:00:00'))

genre_name
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre_name, dtype: int64


In [880]:
print(genre_weekday(moscow_general, 'Friday', '17:00:00', '23:00:00'))

genre_name
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre_name, dtype: int64


In [881]:
print(genre_weekday(spb_general, 'Friday', '17:00:00', '23:00:00'))

genre_name
pop            256
rock           216
electronic     216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre_name, dtype: int64


Популярные жанры в понедельник утром в Питере и Москве оказались похожи: везде, как и предполагалось, популярен pop. Несмотря на это, концовка топ-10 для двух городов различается: в Питере в топ-10 входит джаз и русский рэп, а в Москве жанр world.

В конце недели что в Москве что в Питере в топе все еще поп, но вот в концовке топ-10 в Питере появляется жанр world

## Вывод
Жанр поп безусловный лидер, а топ-5 в целом не различается в обеих столицах. При этом видно, что концовка списка более «живая»: для каждого города выделяются более характерные жанры, которые действительно меняют свои позиции в зависимости от дня недели и времени.


# Москва и Питер — две разные столицы, два разных направления в музыке. Правда?

Гипотеза: Питер богат своей рэп-культурой, поэтому это направление там слушают чаще, а Москва — город контрастов, но основная масса пользователей слушает попсу.

Сгруппируем таблицу moscow_general по жанру, сосчитаем численность композиций каждого жанра методом count(), отсортируем в порядке убывания и сохраним результат в таблице moscow_genres.

Просмотрим первые 10 строк этой новой таблицы.


In [882]:
moscow_genres = moscow_general.groupby(by='genre_name')['genre_name'].count().sort_values(ascending=False)
# одной строкой: группировка таблицы moscow_general по столбцу 'genre_name', 
# подсчёт числа значений 'genre_name' в этой группировке методом count(), 
# сортировка Series в порядке убывания и сохранение в moscow_genres 


In [883]:
moscow_genres.head(10)

genre_name
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre_name, dtype: int64

Сгруппируем таблицу spb_general по жанру, сосчитаем численность композиций каждого жанра методом count(), отсортируем в порядке убывания и сохраним результат в таблице spb_genres.

Просматриваем первые 10 строк этой таблицы. Теперь можно сравнивать два города.


In [884]:
spb_genres = spb_general.groupby(by='genre_name')['genre_name'].count().sort_values(ascending=False)

In [885]:
spb_genres.head(10)

genre_name
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre_name, dtype: int64

## Вывод
В Москве, кроме абсолютно популярного жанра поп, есть направление русской популярной музыки. Значит, что интерес к этому жанру шире. А рэп, вопреки предположению, занимает в обоих городах близкие позиции.


# Результаты исследования

Рабочие гипотезы:

•	музыку в двух городах — Москве и Санкт-Петербурге — слушают в разном режиме;

•	списки десяти самых популярных жанров утром в понедельник и вечером в пятницу имеют характерные отличия;

•	население двух городов предпочитает разные музыкальные жанры.

**Общие результаты**

Москва и Петербург сходятся во вкусах: везде преобладает популярная музыка. При этом зависимости предпочтений от дня недели в каждом отдельном городе нет — люди постоянно слушают то, что им нравится. Но между городами в разрезе дней недели наблюдается зеркальность относительно среды: Москва больше слушает в понедельник и пятницу, а Петербург наоборот - больше в среду, но меньше в понедельник и пятницу.
В результате первая гипотеза подтверждена, вторая гипотеза не подтверждена и третья не подтверждена
